# [SC57 - Working with big, multi-dimensional geoscientific datasets in Python: a tutorial introduction to xarray](http://meetingorganizer.copernicus.org/EGU2017/session/25651)  
  
  
Original notebook by [Stephan Hoyer](http://stephanhoyer.com), Rossbypalooza, 2016.  
Modified by Edward Byers, Matthew Gidden and [Fabien Maussion](http://fabienmaussion.info/) for EGU General Assembly 2017, Vienna, Austria


  
  Thursday, 27th April, 15:30–17:00 / Room -2.91  
  
  
**Convenors**
* [Dr Edward Byers](mailto:byers@iiasa.ac.at)    - International Institute for Applied Systems Analysis, Laxenburg, Austria
* [Dr Matthew Gidden](mailto:gidden@iiasa.ac.at)  - International Institute for Applied Systems Analysis, Laxenburg, Austria
* [Dr Fabien Maussion](mailto:fabien.maussion@uibk.ac.at) - University of Innsbruck, Innsbruck, Austria
-------------


# With

![](./figures/dataset-diagram-logo.png)

# you can reach

![](./figures/facet-plot.png)

# Structure of this tutorial

1. Introduction to key features of `xarray`
2. Basic operations in xarray: opening, inspecting, selecting and indexing data
3. Working with multiple datasets and computation
4. Introduction to out-of-core computation
5. Working with `pandas` and other packages




# 1. Key features of `xarray`

## What is `xarray`?

*  `xarray` is an open source project and Python package
*  `xarray` has been designed to perform **labelled** data analysis on **multi-dimensional** arrays
* the xarray approach adopts the Common Data Model for **self-describing scientific data** in widespread use in the Earth sciences
*  `xarray.Dataset` is an in-memory representation of a netCDF file.
* `xarray` is built on top of the dataprocessing library [Pandas](http://pandas.pydata.org) (the best way to work with tabular data (e.g., CSV files) in Python)

# Our data

<img src="./figures/dataset.png" width="50%" align="right"> 

- numeric
- multi-dimensional
- labelled
- (lots of) metadata
- sometimes (very) large

## What is `xarray` good for?

* Gridded, multi-dimensional and large datasets, commonly used in earth sciences, but also increasingly finance, engineering (signal/image processing), and biological sciences
* Integration with other data analysis packages such as Pandas 
* I/O operations (NetCDF)
* Plotting
* Out of core computation and parallel processing
* Extensions based on xarray
* ...

## Where can I find more info?
This notebook introduces xarray for new users in the geophysical sciences.

### For more information about xarray

- Read the [online documentation](http://xarray.pydata.org/)
- Ask questions on [StackOverflow](http://stackoverflow.com/questions/tagged/python-xarray)
- View the source code and file bug reports on [GitHub](http://github.com/pydata/xarray/)

### For more doing data analysis with Python:

- Thomas Wiecki, [A modern guide to getting started with Data Science and Python](http://twiecki.github.io/blog/2014/11/18/python-for-data-science/)
- Wes McKinney, [Python for Data Analysis](http://shop.oreilly.com/product/0636920023784.do) (book)

### Packages building on xarray for the geophysical sciences

For analyzing GCM output:

- [xgcm](https://github.com/xgcm/xgcm) by Ryan Abernathey
- [oogcm](https://github.com/lesommer/oocgcm) by Julien Le Sommer
- [MPAS xarray](https://github.com/pwolfram/mpas_xarray) by Phil Wolfram
- [marc_analysis](https://github.com/darothen/marc_analysis) by Daniel Rothenberg

Other tools:

- [windspharm](https://github.com/ajdawson/windspharm): wind spherical harmonics by Andrew Dawson
- [eofs](https://github.com/ajdawson/eofs): empirical orthogonal functions by Andrew Dawson
- [infinite-diff](https://github.com/spencerahill/infinite-diff) by Spencer Hill 
- [aospy](https://github.com/spencerahill/aospy) by Spencer Hill and Spencer Clark
- [regionmask](https://github.com/mathause/regionmask) by Mathias Hauser
- [salem](https://github.com/fmaussion/salem) by Fabien Maussion

Resources for teaching and learning xarray in geosciences:
- [Fabien's teaching repo](https://github.com/fmaussion/teaching): courses that combine teaching climatology and xarray


# 2. Basic operations in `xarray`

-------------------

## Import packages

In [ ]:
# standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr

import warnings

%matplotlib inline

np.set_printoptions(precision=3, linewidth=80, edgeitems=1)  # make numpy less verbose
xr.set_options(display_width=70)
warnings.simplefilter('ignore')

## Basic data arrays in numpy

In [ ]:
import numpy as np
a = np.array([[1, 3, 9], [2, 8, 4]])
a

In [ ]:
a[1, 2]

In [ ]:
a.mean(axis=0)

## Properties of `xarray.Dataset` and `xarray.DataArray` objects

We'll start with the "air_temperature" tutorial dataset:

In [ ]:
ds = xr.tutorial.load_dataset('air_temperature')

In [ ]:
ds

In [ ]:
ds.air

In [ ]:
ds.dims

In [ ]:
ds.attrs

In [ ]:
ds.air.values

In [ ]:
type(ds.air.values)

In [ ]:
ds.air.dims

In [ ]:
ds.air.attrs

In [ ]:
ds.air.attrs['tutorial-date'] = 24042017

In [ ]:
ds.air.attrs

## Let's Do Some Math

In [ ]:
kelvin = ds.air.mean(dim='time')
kelvin.plot()

In [ ]:
centigrade = kelvin - 273.16
centigrade.plot()

Notice xarray has changed the colormap according to the dataset (borrowing logic from Seaborn).
* With degrees C, the data passes through 0, so a diverging colormap is used
* With Kelvin, the default colormap is used.

In [ ]:
# ufuncs work too
np.sin(centigrade).plot()

## Adding Data to `DataSet`s

In [ ]:
ds

In [ ]:
ds['centigrade'] = centigrade
ds

# 2. Selecting data with named dimensions

In xarray there are many different ways for selecting and indexing data.

### Positional indexing (old way)

This is the "old way", i.e. like ``numpy``:

In [ ]:
ds.air[:, 1, 2]

In [ ]:
ds.air[:, 1, 2].plot()

This selection implies prior knowledge about the structure of the data, and is therefore much less readable than the "xarray methods" presented below.

### Selection by index

Selection based on the **index** of a coordinate:

In [ ]:
ds.air.isel(time=0).plot()

### Selection by value

Selection based on the **value** of a coordinate:

In [ ]:
ds.air.sel(lat=72.5, lon=205).plot()

### Selection by value works well for time, too

In [ ]:
ds.air.sel(time='2013-01-02').plot() # Note that we will extract 4 time steps!

In [ ]:
ds.air.sel(time='2013-01-02T06:00').plot() # or look at a single timestep

### Selecting a range of values
The syntax is similar, but you'll need to use a [slice](https://docs.python.org/3/library/functions.html#slice):

In [ ]:
ds.air.sel(lat=slice(60, 50), lon=slice(200, 270), time='2013-01-02T06:00:00').plot();

### Nearest neighbor lookup

In [ ]:
ds.air.sel(lat=41.8781, lon=360-87.6298, method='nearest', tolerance=5).plot()

# 3. Operations and computation

* We can do arithmetic directly on `Dataset` and `DataArray` objects. 
* Labels are preserved and dataArray dimensions automatically aligned.

### Broadcasting

<img src="./figures/broadcast.png" width="50%" align="left"> 

In [ ]:
a = xr.DataArray(np.arange(3), dims='time', 
                 coords={'time':np.arange(3)})
b = xr.DataArray(np.arange(4), dims='space', 
                 coords={'space':np.arange(4)})
a + b

### Alignment

<img src="./figures/align.png" width="50%" align="left"> 

In [ ]:
atime = np.arange(3)
btime = np.arange(5) + 1
atime, btime

In [ ]:
a = xr.DataArray(np.arange(3), dims='time', 
                 coords={'time':atime})
b = xr.DataArray(np.arange(5), dims='time', 
                 coords={'time':btime})
a + b

### Aggregation


In [ ]:
ds.max()

In [ ]:
ds.air.median(dim=['lat', 'lon']).plot()

### Masking with `.where()`

In [ ]:
means = ds.air.mean(dim=['time'])
means.where(means > 273.15).plot()

### Dealing with Outliers

In [ ]:
air_outliers = ds.air.isel(time=0).copy()
air_outliers[0, 0] = 100
air_outliers[-1, -1] = 400
air_outliers.plot()

In [ ]:
air_outliers.plot(robust=True)

Using `robust=True` uses the 2nd and 98th percentiles of the data to compute the color limits.

# 4. Groupby and "split-apply-combine"

Xarray implements the "split-apply-combine" paradigm with `groupby`. This works really well for calculating climatologies:

In [ ]:
ds.air.groupby('time.season').mean()

In [ ]:
ds.air.groupby('time.month').mean('time') #.mean()

In [ ]:
clim = ds.air.groupby('time.month').mean('time')

You can also do arithmetic with groupby objects, which repeats the arithmetic over each group:

In [ ]:
anomalies = ds.groupby('time.month') - clim

In [ ]:
anomalies

In [ ]:
anomalies.air.plot() 

In [ ]:
anomalies.air.sel(time= '2013-02').plot() # Find all the values for February

Resample adjusts a time series to a new resolution:

In [ ]:
tmin = ds.air.resample('1D', dim='time', how='min')  # Resample to one day '1D
tmax = ds.air.resample('1D', dim='time', how='max')

In [ ]:
tmin

In [ ]:
ds_extremes = xr.Dataset({'tmin': tmin, 'tmax': tmax})

In [ ]:
ds_extremes

# 5. Graphics 

Examples of graphics (1D, 2D - contour, Facet plots)

In [ ]:
f = 'ERA-Int-MonthlyAvg-4D-TUVWZ.nc'
dse = xr.open_dataset(f)
dse

In [ ]:
dse.u.mean(dim=['month', 'longitude']).plot.contourf(levels=13)
plt.ylim([1000, 100]);

In [ ]:
u_avg = dse.u.mean(dim=['month', 'longitude'])
u_avg_masked = u_avg.where(u_avg > 12)
u_avg_masked.plot.contourf(levels=13)
plt.ylim([1000, 100]);

### Plotting on maps

In [ ]:
import cartopy.crs as ccrs

In [ ]:
f = plt.figure(figsize=(12, 4))
# Define the map projection of the plots
ax = plt.axes(projection=ccrs.PlateCarree())
# ax is an empty plot. We now plot the variable sw_avg onto ax
ds.air.mean(dim='time').plot(ax=ax, transform=ccrs.PlateCarree()) 
# the keyword "transform" tells the function in which projection the data is stored 
ax.coastlines(); ax.gridlines(); # Add gridlines and coastlines to the plot

In [ ]:
ax = plt.axes(projection=ccrs.Orthographic(-80, 35))
ds.air.isel(time=0).plot.contourf(ax=ax, transform=ccrs.PlateCarree());
ax.set_global(); ax.coastlines();

### Seaborn is Cool

Statistical visualization with [Seaborn](https://stanford.edu/~mwaskom/software/seaborn/):

In [ ]:
import seaborn as sns

data = (ds_extremes
        .sel_points(lat=[41.8781, 37.7749], lon=[360-87.6298, 360-122.4194],
                    method='nearest', tolerance=3,
                    dim=xr.DataArray(['Chicago', 'San Francisco'],
                                     name='location', dims='location'))
        .to_dataframe()
        .reset_index()
        .assign(month=lambda x: x.time.dt.month))

plt.figure(figsize=(10, 5))
sns.violinplot('month', 'tmax', 'location', data=data, split=True, inner=None)

# 6. xarray also works for data that doesn't fit in memory

Here's a quick demo of [how xarray can leverage dask](http://xarray.pydata.org/en/stable/dask.html) to work with data that doesn't fit in memory. This lets xarray substitute for tools like `cdo` and `nco`.

Tell dask we want to use 4 threads (one for each core we have):

In [ ]:
import dask
from multiprocessing.pool import ThreadPool

dask.set_options(pool=ThreadPool(4))

In [ ]:
dsdask = xr.open_dataset('ERA-Interim-MonthlyAvg-TUVP.nc').chunk({'time': 100})
dsdask.data

In [ ]:
%time ds_seasonal.load()

In [ ]:
%time ds_seasonal = ds.groupby('time.season').mean('time')

In [ ]:
dask.set_options(pool=ThreadPool(16))

In [ ]:
%time ds_month = ds.groupby('time.season').mean('time')

In [ ]:
(ds_seasonal['air']
 .sel(season=['DJF', 'MAM', 'JJA', 'SON'])
 .plot(col='season', size=3, cmap='Spectral_r'))

For more details, read this blog post: http://continuum.io/blog/xray-dask